# The Palatable Diet Problem  

**Problem Description.** The Diet Problem is the first large-scale optimization problem to be solved with the Simplex algorithm by Jack Laderman in [1947](https://www.mpi-inf.mpg.de/fileadmin/inf/d1/teaching/winter18/Ideen/Materialien/Dantzig-Diet.pdf). The basic formulation of this problem consists of minimizing the cost of a food basket while meeting the specified nutrient requirements. In this notebook, we solve The Palatable Diet Problem (TPDP), where the basic model is extended with a constraint on the food basket palatability. An explicit formula of the palatability constraint is unknown, but we have data on several food baskets and the respective palatability score. First, we define a conceptual model with the *known constraint*. Then, OptiCL is used to learn and embed the palatability constraint.  
(*TPDP is part of a larger optimization problem which simultaneously  optimizes  the  food  basket  to  be  delivered,  the  sourcing  plan,  the  delivery  plan, and  the  transfer  modality  of  a  month-long  food  supply in a Wolrd Food Program setting ([Maragno et al., 2021])*).

<font color='#808080'>**Objective function:** minimize the total cost of the food basket.</font>  
$\min_{\boldsymbol{x}} c^\top \boldsymbol{x}$

*subject to* 

<font color='#808080'>**Nutritional constraints:** for each nutrient $l\in\mathcal{L}$, at least meet the minimum required level.</font>  
$ \sum_{k \in \mathcal{K}} nutval_{kl} x_{k} \geq nutreq_{l}, \ \ \ \forall l\in\mathcal{L},$   
<font color='#808080'>**Constraints on sugar and salt.**</font>  
$ x_{salt} = 5,$   
$ x_{sugar} = 20,$  
<font color='#808080'>**Palatability constraints:** the food basket palatability has to be at least equal to $t$.</font>  
$ y \geq t,$  
<font color='#808080'>**Learned predictive model:** the palatability is defined using a predictive model.</font>  
$ y = \hat{h}(\boldsymbol{x}),$   
<font color='#808080'>**Non negativity constraints.**</font>  
$ x_{k} \geq 0, \ \ \ \forall k \in \mathcal{K}.$  

In [2]:
import pandas as pd
from imp import reload
import numpy as np
import math
from sklearn.utils.extmath import cartesian
import time
import sys
import os
sys.path.append(os.path.abspath('../../src'))  # TODO: has to be changed
import opticl
from pyomo import environ
from pyomo.environ import *
np.random.seed(0)

### Data Loading  
**nutr_val**: nutritional values for each of the 25 foods  
**nutr_req**: 11 nutrition requirements  
**cost_p**: vector of procurement costs  
**dataset**: dataframe of food basket instances and relative palatability score

In [214]:
nutr_req

,Energy(kcal),Protein(g),Fat(g),Calcium(mg),Iron(mg),VitaminA(ug),ThiamineB1(mg),RiboflavinB2(mg),NicacinB3(mg),Folate(ug),VitaminC(mg),Iodine(ug)
Type,,,,,,,,,,,,
Avg person day,2100,52.5,35,1100,22,500,0.9,1.4,12,160,28,150


In [238]:
nutr_val = pd.read_excel('processed-data/Syria_instance.xlsx', sheet_name='nutr_val', index_col='Food')
nutr_req = pd.read_excel('processed-data/Syria_instance.xlsx', sheet_name='nutr_req', index_col='Type')
cost_p = pd.read_excel('processed-data/Syria_instance.xlsx', sheet_name='FoodCost', index_col='Supplier').iloc[0,:]
dataset = pd.read_csv('processed-data/WFP_dataset.csv').sample(frac=1)

nutr_val.drop(['Salt','Sugar'], axis=0, inplace = True)
nutr_val.drop(['Iodine(ug)'], axis=1, inplace = True)
nutr_req.drop(['Iodine(ug)'], axis=1, inplace = True)
dataset.drop(['Salt','Sugar'], axis=1, inplace=True)

In [ ]:
def normalize(y):
    minimum = 71.969  
    maximum = 444.847  
    return 1 - (y - minimum)/(maximum - minimum)

def check_violation(threshold, solution):
    # Cereals & Grains
    group1 = [1, 11, 12, 14, 15, 22, 23]
    group1_names = [list(solution.keys())[i] for i in group1]
    values1 = [solution[x] for x in group1_names]
    food_in_group1 = sum(values1)*100
#     print(f'food_in_group1 {food_in_group1}')
    idealG1 = 400
    distG1 = food_in_group1 - idealG1
    
    # Pulses & Vegetables
    group2 = [0, 6, 10, 13]
    group2_names = [list(solution.keys())[i] for i in group2]
    values2 = [solution[x] for x in group2_names]
    food_in_group2 = sum(values2)*100
#     print(f'food_in_group2 {food_in_group2}')
    idealG2 = 65
    distG2 = food_in_group2 - idealG2
    
    # Oils & Fats
    group3 = [21]
    group3_names = [list(solution.keys())[i] for i in group3]
    values3 = [solution[x] for x in group3_names]
    food_in_group3 = sum(values3)*100
#     print(f'food_in_group3 {food_in_group3}')
    idealG3 = 27
    distG3 = food_in_group3 - idealG3
    
    # Mixed & Blended Foods
    group4 = [5, 24, 16, 17, 18, 19]
    group4_names = [list(solution.keys())[i] for i in group4]
    values4 = [solution[x] for x in group4_names]
    food_in_group4 = sum(values4)*100
#     print(f'food_in_group4 {food_in_group4}')
    idealG4 = 45
    distG4 = food_in_group4 - idealG4
    
    # Meat & Fish & Dairy
    group5 = [2, 3, 4, 7, 8]
    group5_names = [list(solution.keys())[i] for i in group5]
    values5 = [solution[x] for x in group5_names]
    food_in_group5 = sum(values5)*100
#     print(f'food_in_group5 {food_in_group5}')
    idealG5 = 30
    distG5 = food_in_group5 - idealG5
    
    real_palatability = np.round(math.sqrt(distG1 ** 2 + (5.7 * distG2) ** 2 + (16.6 * distG3) ** 2
                                      + (4.4 * distG4) ** 2 + (6.6 * distG5) ** 2), 3)
    real_palatability_norm = normalize(real_palatability)
    
    return 1-int(real_palatability_norm>=threshold), real_palatability_norm

# OptiCL: Optimization with Constraint Learning

## Step 1: Conceptual Model

In [239]:
def init_conceptual_model(cost_p):
    N = list(nutr_val.index)  # foods
    M = nutr_req.columns  # nutrient requirements

    model = ConcreteModel('TPDP')

    '''
    Decision variables
    '''
    model.x = Var(N, domain=NonNegativeReals)  # variables controlling the food basket

    '''
    Objective function.
    '''
    def obj_function(model):
        return sum(cost_p[food].item()*model.x[food] for food in N)

    model.OBJ = Objective(rule=obj_function, sense=minimize)

    '''
    Nutrients requirements constraint.
    '''
    def constraint_rule1(model, req):
        return sum(model.x[food] * nutr_val.loc[food, req] for food in N) >= nutr_req[req].item()
    model.Constraint1 = Constraint(M, rule=constraint_rule1)
    '''
    Sugar constraint
    '''
#     def constraint_rule2(model):
#         return model.x['Sugar'] == 0.2
#     model.Constraint2 = Constraint(rule=constraint_rule2)
#     '''
#     Salt constraint
#     '''
#     def constraint_rule3(model):
#         return model.x['Salt'] == 0.05
#     model.Constraint3 = Constraint(rule=constraint_rule3)
    
    return model

## Step 2: Data Processing

In [240]:
y = dataset['label']
X = dataset.drop(['label'], axis=1, inplace=False)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

Check ground truth function.

In [241]:
# y_true = [check_violation(0.5, X_train.iloc[i,:])[1] for i in range(X_train.shape[0])]
# y_compare = pd.DataFrame({'y_train':y_train,
#     'y_calc':y_true, 
#     'diff':abs(y_train - y_true),
#     'match':(np.round(y_true,2)==np.round(y_train, 2))})

## Part 3: Learn the predictive models
'alg_list' specifies the list of algorithms that you will consider in the training pipeline. If you have the InterpretableAI license, you can include **iai** (Optimal Trees with Hyperplanes) or **iai-single** (Optimal Trees with single feature splits) in the list. If using IAI, you must specify the metric as 'r2'. Otherwise, the default metric is 'neg_squared_mse'.

#### Goldfarb and Iyengar: closed-form solution

In [242]:
A = X_train
A_cols = A.columns
A = np.matrix(A)

In [243]:
## OLS estimator
B_hat = ((A.T @ A).I @ A.T) @ np.matrix(y_train).T
resids = np.matrix(y_train).T - (A @ B_hat)
n_features = A.shape[1] # m+1 in iyengar goldfarb
n_samples = A.shape[0] # p in IG
σ_sq = np.square(resids).sum() / (n_samples - n_features) # square of L2 norm / (num samples - num features)

In [244]:
from scipy.stats import f

In [245]:
f_dist = f(dfn = A.shape[1], dfd = A.shape[0] - A.shape[1])
w_crit = f_dist.ppf(q = 0.95)

In [246]:
B_temp = B_hat + 0.001

In [247]:
## P() = w (.95)
(B_temp - B_hat).T @ (A.T @ A) @ (B_temp - B_hat) <= n_features*w_crit*σ_sq

matrix([[ True]])

In [250]:
N = X_train.columns
Q = pd.DataFrame((A.T  @ A).I, columns = N, index = N)

In [280]:
 np.all(np.linalg.eigvals((A.T  @ A).I) > 0)

True

In [256]:
model.x['Beans']

In [289]:
np.random.seed(0)
price_random = pd.Series(np.random.random(len(cost_p))*1000)
price_random.index = cost_p.index

conceptual_model = init_conceptual_model(price_random)
model_master = pd.DataFrame()
model = opticl.optimization_MIP(conceptual_model, conceptual_model.x, model_master, X_train, 
                                          tr=True)

## Inputs into function
outcome = 'palatability'
threshold = 0.5;
lb = threshold; ub = np.nan;
weight_objective = 0; 
task = 'continuous'
data = X_train
Γ = 0.5

## Inner function to embed constraint
N = X_train.columns


coeff = pd.DataFrame(B_hat.T, columns = N).loc[0,:]

model.add_component('LR'+outcome, 
                    Constraint(expr=model.y[outcome] == sum(model.x[i] * coeff.loc[i] for i in N)))

model.add_component('lb_' + outcome, Constraint(expr=model.y[outcome] - np.sqrt(Γ)*sum(model.x[i]*Q.loc[i,j]*model.x[j] for i in N for j in N) <= lb))

# model.write('test_robust_cf.lp')

Generating constraints for the trust region using 1000 samples.
... Trust region defined.


In [290]:
opt = SolverFactory('gurobi')
# opt.options['NonConvex'] = 2
start_time = time.time()
results = opt.solve(model) 
computation_time = time.time() - start_time
pred_palat = value(model.y['palatability'])
# violation_bool, real_palat = check_violation(threshold,  model.x.get_values())

ERROR: Solver (gurobi) returned non-zero return code (1)
ERROR: Solver log: Academic license - for non-commercial use only - expires
    2022-08-14 Using license file /Users/hollywiberg/gurobi.lic Read LP format
    model from file
    /var/folders/n9/m94rpfdx0r1d_ltdt2pgzwy80000gn/T/tmpcnncmcf_.pyomo.lp
    Reading time = 0.01 seconds x1025: 37 rows, 1025 columns, 8121 nonzeros
    Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64) Thread count: 4
    physical cores, 8 logical processors, using up to 8 threads Optimize a
    model with 37 rows, 1025 columns and 8121 nonzeros Model fingerprint:
    0x1ecb7a38 Model has 1 quadratic constraint Coefficient statistics:
      Matrix range     [5e-05, 2e+03] QMatrix range    [5e-05, 2e+00] QLMatrix
      range   [1e+00, 1e+00] Objective range  [2e+01, 1e+03] Bounds range
      [1e+00, 1e+00] RHS range        [9e-01, 2e+03] QRHS range       [5e-01,
      5e-01]
    Presolve removed 1 rows and 1 columns Traceback (most recent call last):
 

ApplicationError: Solver (gurobi) did not exit normally

In [5]:
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.tree import DecisionTreeRegressor, plot_tree

### Bootstrapped intervals

In [291]:
from sklearn.utils import resample
from scipy.stats import bootstrap

from julia import Julia
Julia(compiled_modules=False)
from interpretableai import iai

In [296]:
gs.get_grid_results()

,minbucket,cp,train_score,valid_score,rank_valid_score
0,5,0.011382,0.677940,0.401704,1
1,10,0.000927,0.673372,0.387928,2


#### IAI with hyperplanes

In [303]:
# results = []
results_nosparse = []
n_resample = 5
for s in range(n_resample):
    print(s)
    X_sub, y_sub = resample(X_train, y_train, n_samples = int(.5*X_train.shape[0]), random_state = s)
    est = iai.OptimalTreeRegressor(
                random_seed=1,
                ls_num_hyper_restarts = 5, # 5 is default
                fast_num_support_restarts = 10,
                hyperplane_config={'sparsity':'all'},
                max_depth = 2,
            )
    gs = iai.GridSearch(est, minbucket = [5,10])              
    gs.fit(X_sub, y_sub)
    grid_result = gs.get_grid_results()
    valid_score = grid_result.query('rank_valid_score == 1')['valid_score'].values[0]
    best_params = gs.get_best_params()
    param_grid = {'minbucket': grid_result['minbucket'].unique(),
    'max_depth':2}
    model = gs.get_learner()
    results_nosparse.append(model)
        

0
1
2
3
4


In [304]:
for s in range(n_resample):
    results_nosparse[s].show_in_browser()

#### Linear regression (no regularization)

In [27]:
results = []
n_resample = 100
for s in range(n_resample):
    X_sub, y_sub = resample(X_train, y_train, n_samples = int(.5*X_train.shape[0]), random_state = s)
    est = LinearRegression()
    est.fit(X_sub, y_sub)
    results.append(est)
coefs = pd.DataFrame(np.stack([results[s].coef_ for s in range(n_resample)]))
coefs.columns = X_train.columns
coefs['intercept'] = [results[s].intercept_ for s in range(n_resample)]

In [35]:
coefs.median(axis=0)

Beans                          -1.723183e-01
Bulgur                         -5.785947e-02
Cheese                         -8.094977e-01
Fish                           -5.216572e-01
Meat                           -9.716336e-01
CSB                            -1.568614e+00
Dates                          -6.459638e-02
DSM                            -4.171014e-01
Milk                           -4.178734e-01
Salt                           -4.336809e-16
Lentils                        -1.400356e-01
Maize                          -6.333926e-02
Maize meal                     -6.472050e-02
Chickpeas                      -9.967434e-02
Rice                           -6.283761e-02
Sorghum                        -6.427730e-02
Soya-fortified bulgur wheat    -1.434997e+00
Soya-fortified maize meal      -1.926228e+00
Soya-fortified sorghum grits   -8.655736e-01
Soya-fortified wheat flour     -1.194380e+00
Sugar                          -8.326673e-17
Oil                            -5.430939e-01
Wheat     

In [82]:
coefs_ci = pd.DataFrame(columns=coefs.columns)
coefs_ci.loc['median',:] = coefs.median(axis=0)
coefs_ci.loc['lb_ci95',:] = np.percentile(coefs,2.5,axis=0)
coefs_ci.loc['ub_ci95',:] = np.percentile(coefs,97.5,axis=0)
coefs_ci.loc['midpoint',:] = (coefs_ci.loc['ub_ci95',:]+coefs_ci.loc['lb_ci95',:])/2
coefs_ci.loc['interval',:] = (coefs_ci.loc['ub_ci95',:]-coefs_ci.loc['midpoint',:])

In [127]:
np.random.seed(0)
price_random = pd.Series(np.random.random(len(cost_p))*1000)
price_random.index = cost_p.index

conceptual_model = init_conceptual_model(price_random)
model_master = pd.DataFrame()
model = opticl.optimization_MIP(conceptual_model, conceptual_model.x, model_master, X_train, 
                                          tr=True)

## Inputs into function
coefficients = coefs_ci
outcome = 'palatability'
threshold = 0.5;
lb = threshold; ub = np.nan;
weight_objective = 0; 
task = 'continuous'
data = X_train
Γ = 0.5

## Inner function to embed constraint
N = data.columns

intercept = coefficients['intercept'][0]
intercept_u = coefficients['intercept'][0]
coeff = coefficients.drop(['intercept'], axis=1, inplace=False).loc['midpoint', :]
coeff_u = coefficients.drop(['intercept'], axis=1, inplace=False).loc['interval', :]
# model.add_component('LR'+outcome, 
#                     Constraint(expr=model.y[outcome] == sum(model.x[i] * coeff.loc[i] for i in N) + intercept))
model.add_component('LR'+outcome, 
                    Constraint(expr=model.y[outcome] == sum(model.x[i] * coeff.loc[i] for i in N) + intercept
                              + Γ*(1/len(N)+1)*sum((model.x[i]*model.x[i] * coeff_u.loc[i]**2) for i in N)))

model.add_component('lb_' + outcome, Constraint(expr=model.y[outcome] >= lb))

model.write('test_robust.lp')

Generating constraints for the trust region using 1000 samples.
... Trust region defined.


('test_robust.lp', 140275138601296)

In [128]:
    
opt = SolverFactory('gurobi')
opt.options['NonConvex'] = 2
start_time = time.time()
results = opt.solve(model) 
computation_time = time.time() - start_time
pred_palat = value(model.y['palatability'])
violation_bool, real_palat = check_violation(threshold,  model.x.get_values())



In [129]:
pred_palat

0.5

In [130]:
real_palat

0.7183851018295528

#### CART

In [122]:
results_shallow = []
constraints_shallow = []
alg = 'cart'
task_type = 'continuous'
outcome = 'temp'

for s in range(10):
    X_sub, y_sub = resample(X_train, y_train, n_samples = int(.5*X_train.shape[0]), random_state = s)
    m, perf = opticl.run_model(X_sub, y_sub, X_test, y_test, alg, outcome, task = task_type,
                       seed = s, cv_folds = 5, 
                       parameter_grid = {"max_depth": [1,2],
                                'min_samples_leaf': [0.02, 0.04, 0.06],
                                "max_features": [0.4, 0.6, 0.8, 1.0]},
                       save = False,
                      )
    constraintL = opticl.ConstraintLearning(X_sub, y_sub, m, alg)
    constraint_add = constraintL.constraint_extrapolation(task_type)
    constraint_add['sample'] = s
    results_shallow.append(m)
    constraints_shallow.append(constraint_add)
    
    plt.figure(figsize = (20,10))
    plot_tree(results_shallow[s],
                   filled=True,feature_names = train_x.columns, fontsize = 12)  
    plt.savefig('results/temp_tree%d.png' % s,format='png',bbox_inches = "tight")

------------- Initialize grid  ----------------
------------- Running model  ----------------
Algorithm = cart, metric = None
------------- Model evaluation  ----------------
-------------------training evaluation-----------------------
Train MSE: 0.02577521263180475
Train R2: 0.5068797672428444
-------------------testing evaluation-----------------------
Test MSE: 0.026921386672781992
Test R2: 0.4741360985468972


NameError: name 'plt' is not defined

#### MLP

In [352]:
results_mlp = []
constraints_mlp = []
alg = 'mlp'
task_type = 'continuous'
outcome = 'temp'

for s in range(25):
    X_sub, y_sub = X_train, y_train
#     X_sub, y_sub = resample(X_train, y_train, n_samples = int(.5*X_train.shape[0]), random_state = s)
    m, perf = opticl.run_model(X_sub, y_sub, X_test, y_test, alg, outcome, task = task_type,
                       seed = s, cv_folds = 5, 
                       parameter_grid = {'hidden_layer_sizes': [(5,)]},
                       save = False,
                      )
    constraintL = opticl.ConstraintLearning(X_sub, y_sub, m, alg)
    constraint_add = constraintL.constraint_extrapolation(task_type)
    constraint_add['sample'] = s
    results_mlp.append(m)
    constraints_mlp.append(constraint_add)

------------- Initialize grid  ----------------
------------- Running model  ----------------
Algorithm = mlp, metric = None
------------- Model evaluation  ----------------
-------------------training evaluation-----------------------
Train MSE: 0.04404702130396748
Train R2: 0.1007471564623974
-------------------testing evaluation-----------------------
Test MSE: 0.0491405592406819
Test R2: 0.04988159748587273
------------- Initialize grid  ----------------
------------- Running model  ----------------
Algorithm = mlp, metric = None
------------- Model evaluation  ----------------
-------------------training evaluation-----------------------
Train MSE: 0.04896734120432957
Train R2: 0.00029515016255898807
-------------------testing evaluation-----------------------
Test MSE: 0.05536235057940182
Test R2: -0.07041492617735234
------------- Initialize grid  ----------------
------------- Running model  ----------------
Algorithm = mlp, metric = None
------------- Model evaluation  -------

------------- Model evaluation  ----------------
-------------------training evaluation-----------------------
Train MSE: 0.0342309566840877
Train R2: 0.30114944838720137
-------------------testing evaluation-----------------------
Test MSE: 0.037011821930651376
Test R2: 0.28438720132075057
------------- Initialize grid  ----------------
------------- Running model  ----------------
Algorithm = mlp, metric = None
------------- Model evaluation  ----------------
-------------------training evaluation-----------------------
Train MSE: 0.05337470716195485
Train R2: -0.08968451821398049
-------------------testing evaluation-----------------------
Test MSE: 0.05721213136852299
Test R2: -0.10617989905348812
------------- Initialize grid  ----------------
------------- Running model  ----------------
Algorithm = mlp, metric = None
------------- Model evaluation  ----------------
-------------------training evaluation-----------------------
Train MSE: 0.055900208185630476
Train R2: -0.14124450

In [353]:
df_mlp = pd.concat(constraints_mlp)
df_mlp['id'] = df_mlp.apply(lambda row: '%d_%d' % (row['sample'],row['node']), axis = 1)

In [354]:
df_clust = df_mlp.query('layer == 0').drop(columns=['layer','node','sample'])
df_clust.set_index('id', inplace = True)

In [355]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import StandardScaler


df_clust_std = StandardScaler().fit_transform(df_clust)

distortions = []
sm = []
K = range(2,20)
for k in K:
    km = KMeans(n_clusters=k)
    km.fit(df_clust_std)
    distortions.append(km.inertia_)
    sm.append(silhouette_score(df_clust_std, km.labels_))

In [356]:
km_final = KMeans(n_clusters=5).fit(df_clust_std)

In [357]:
df_clust['cluster'] = km_final.labels_

In [358]:
km_final.labels_

array([2, 4, 0, 2, 4, 2, 3, 2, 2, 1, 1, 0, 3, 2, 2, 2, 0, 2, 2, 4, 0, 0,
       0, 2, 0, 0, 0, 3, 1, 3, 1, 3, 3, 1, 0, 3, 1, 2, 1, 1, 0, 4, 0, 2,
       1, 3, 1, 3, 3, 3, 3, 1, 3, 0, 4, 0, 3, 2, 0, 4, 3, 4, 2, 2, 2, 1,
       2, 0, 3, 2, 3, 3, 1, 2, 3, 2, 0, 2, 1, 1, 2, 1, 4, 2, 2, 4, 0, 1,
       1, 1, 2, 1, 1, 1, 0, 2, 1, 0, 1, 1, 1, 4, 1, 0, 3, 3, 3, 3, 0, 1,
       1, 2, 0, 2, 1, 1, 3, 0, 2, 3, 1, 1, 4, 2, 3], dtype=int32)

In [359]:
df_clust

,intercept,node_0,node_1,node_2,node_3,node_4,node_5,node_6,node_7,node_8,...,node_14,node_15,node_16,node_17,node_18,node_19,node_20,node_21,node_22,cluster
id,,,,,,,,,,,,,,,,,,,,,
0_0,-0.192090,-0.051330,0.038285,0.159164,-0.486206,0.370750,-0.009431,-0.284960,0.033945,-0.237019,...,0.286745,-0.599725,-0.300206,-0.359261,-0.304104,-0.379760,0.037109,-0.001645,-0.140200,2
0_1,0.196546,0.070352,0.287314,0.739568,0.283704,0.716930,0.355266,0.261215,-0.307240,0.136811,...,0.285664,0.453056,0.871517,0.291878,0.853912,-0.026116,0.152293,-0.263396,-0.012205,4
0_2,0.021915,0.059912,0.199503,-0.236768,0.033368,-0.376823,0.152468,-0.119253,0.129447,0.063120,...,0.287425,-0.600685,-0.653781,-0.598056,-0.652159,-0.504777,0.065649,0.203845,0.173537,0
0_3,0.319920,0.018348,0.285793,0.137375,-0.051267,0.087529,-0.240628,0.004205,0.455685,-0.523829,...,-0.038386,-0.476996,-0.190153,-0.096805,-0.091453,0.264889,0.307436,-0.237137,-0.169341,2
0_4,0.101291,-0.150717,0.212394,0.263515,0.886851,0.304279,0.471611,-0.408317,-0.193947,0.226614,...,0.423816,0.100878,0.556217,0.718752,0.448410,-0.507323,0.041455,0.275261,0.148278,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24_0,-0.028950,0.365894,0.376940,-0.439462,-0.488145,0.077992,0.238954,0.077148,0.261453,-0.256609,...,0.026988,0.160036,-0.460461,-0.047585,-0.251922,-0.255886,0.088210,-0.099144,0.331491,1
24_1,-0.185351,0.043143,0.301657,0.034122,0.407265,0.038671,0.343987,0.176164,0.119126,0.088885,...,0.181652,0.356734,0.000460,-0.416271,0.148548,-0.053905,0.127380,-0.070899,-0.247296,1
24_2,-0.405942,0.314824,0.082223,0.593928,0.445128,0.391151,0.294220,0.323016,-0.203816,-0.072727,...,0.406456,-0.037521,0.294922,0.657125,0.574307,0.066674,0.066188,-0.411847,0.236671,4


In [367]:
df_clust.to_csv('results/mlp_constraints_bootstrapped_fulldata.csv', index = True)

## Step 4: Predictive model selection and Optimization

In [67]:
threshold = 0.5
trust_region = True

In [363]:
version = 'TPDP_v1'
outcome_list = ['palatability']
constraints_embed = ['palatability']
objectives_embed = {}
performance = pd.read_csv('results/%s_performance.csv' % version)
performance.dropna(axis='columns')
performance['task'] = task

In [364]:
performance

,save_path,seed,cv_folds,parameters,best_params,valid_score,train_score,train_r2,test_score,test_r2,auc_train,auc_test,outcome,alg,task
0,results/mlp/TPDP_v1_palatability_model.csv,1,5,"{'hidden_layer_sizes': [(10,), (20,), (50,), (...","{'hidden_layer_sizes': (100,)}",-0.019855,0.011065,0.783251,0.018395,0.641899,NaN,NaN,palatability,mlp,continuous
1,results/linear/TPDP_v1_palatability_model.csv,1,5,"{'alpha': [0.1, 1, 10, 100, 1000], 'l1_ratio':...","{'alpha': 0.1, 'l1_ratio': 0.1}",-0.045594,0.044942,0.119613,0.046470,0.095323,NaN,NaN,palatability,linear,continuous
2,results/cart/TPDP_v1_palatability_model.csv,1,5,"{'max_depth': [3, 4, 5, 6, 7, 8, 9, 10], 'min_...","{'max_depth': 10, 'max_features': 1.0, 'min_sa...",-0.016014,0.010676,0.790856,0.015873,0.690982,NaN,NaN,palatability,cart,continuous
3,results/rf/TPDP_v1_palatability_model.csv,1,5,"{'n_estimators': [10, 25], 'max_features': ['a...","{'max_depth': 4, 'max_features': 'auto', 'n_es...",-0.016288,0.013386,0.737770,0.016551,0.677784,NaN,NaN,palatability,rf,continuous
4,results/svm/TPDP_v1_palatability_model.csv,1,5,"{'C': [0.1, 1, 10, 100]}",{'C': 100},-0.019034,0.018145,0.644559,0.019587,0.618690,NaN,NaN,palatability,svm,continuous
5,results/gbm/TPDP_v1_palatability_model.csv,1,5,"{'learning_rate': [0.01, 0.025, 0.05, 0.075, 0...","{'learning_rate': 0.2, 'max_depth': 5, 'n_esti...",-0.008328,0.003146,0.938371,0.008673,0.831147,NaN,NaN,palatability,gbm,continuous


In [365]:
model_master = opticl.model_selection(performance.query('alg == "%s"' % 'rf'), 
                                      constraints_embed, objectives_embed)

        outcome model_type                                  save_path  \
0  palatability         rf  results/rf/TPDP_v1_palatability_model.csv   

         task  objective  
0  continuous          0  


In [366]:
model_master

,outcome,model_type,save_path,task,objective
0,palatability,rf,results/rf/TPDP_v1_palatability_model.csv,continuous,0


In [128]:
columns_df = ['algorithm','iteration','price_matrix']+list(X.columns)+['objective_function', 'real_palat', 'pred_palat', 'violation', 'time']
solutions_df = pd.DataFrame(columns = columns_df)
iterations = 25  # 1000

for alg in alg_list:
    print(f"\nAlgorithm: {alg}")
    
    model_master = opticl.model_selection(performance.query('alg == "%s"' % alg), 
                                      constraints_embed, objectives_embed)
    model_master['lb'] = threshold
    model_master['ub'] = None
    opticl.check_model_master(model_master)

    for i in range(iterations):
        print(f"Iteration {i}")
        np.random.seed(i)
        price_random = pd.Series(np.random.random(len(cost_p))*1000)
        price_random.index = cost_p.index

        conceptual_model= init_conceptual_model(price_random)
        MIP_final_model = opticl.optimization_MIP(conceptual_model, conceptual_model.x, model_master, X, 
                                                  tr=True)
        opt = SolverFactory('gurobi')
        start_time = time.time()
        results = opt.solve(MIP_final_model) 
        computation_time = time.time() - start_time
        pred_palat = value(MIP_final_model.y['palatability'])
        violation_bool, real_palat = check_violation(threshold,  MIP_final_model.x.get_values())
        ## Save solutions
        solution = MIP_final_model.x.get_values()
        solution['algorithm'] = alg
        solution['iteration'] =i
        solution['price_matrix'] = price_random.to_dict()
        solution['violation'] = violation_bool
        solution['real_palat'] = real_palat
        solution['pred_palat'] = pred_palat
        solution['objective_function'] = value(MIP_final_model.OBJ)
        solution['time'] = computation_time
        solutions_df = solutions_df.append(solution, ignore_index=True)
        print(f"The predicted palatability of the optimal solution is {pred_palat}")


Algorithm: linear
        outcome model_type                                      save_path  \
0  palatability     linear  results/linear/TPDP_v1_palatability_model.csv   

         task  objective  
0  continuous          0  
No learned objective

Embedding constraint for palatability using linear model.
0.5 <= palatability
Iteration 0
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.5
Iteration 1
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.5
Iteration 2
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.5263742138343516
Iteration 3
Generating constraints for the trust region usin

... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.5527527524722223
Iteration 12
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.7453189247368421
Iteration 13
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.5527527524722223
Iteration 14
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.6602250677727274
Iteration 15
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.5497993538076923
It

... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.5278823759952237
Iteration 24
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.6619847689686196

Algorithm: svm
        outcome model_type                                   save_path  \
0  palatability        svm  results/svm/TPDP_v1_palatability_model.csv   

         task  objective  
0  continuous          0  
No learned objective

Embedding constraint for palatability using svm model.
0.5 <= palatability
Iteration 0
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.5
Iteration 1
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for pala

The predicted palatability of the optimal solution is 0.5036995957817828
Iteration 10
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.5007872332617097
Iteration 11
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.6230645976112993
Iteration 12
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.594452470583912
Iteration 13
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.516831970069418
Iteration 14
Generating constraints for the trust region using 5000 s

In [129]:
solutions_df_robust = solutions_df.copy()

In [130]:
solutions_df_robust['max_violation'] = np.nan

In [134]:
columns_df = ['algorithm','iteration','price_matrix']+list(X.columns)+['objective_function', 'real_palat', 'pred_palat', 'violation', 'time']
solutions_df = pd.DataFrame(columns = columns_df)
iterations = 25  # 1000
alg = 'linear'
for Γ in [0.5, 1, 2]:
    for i in range(iterations):
            print(f"Iteration {i}")
            np.random.seed(i)
            price_random = pd.Series(np.random.random(len(cost_p))*1000)
            price_random.index = cost_p.index
            conceptual_model = init_conceptual_model(price_random)


            model_master = pd.DataFrame()
            model = opticl.optimization_MIP(conceptual_model, conceptual_model.x, model_master, X_train, 
                                              tr=True)


            ## Inputs into function
            coefficients = coefs_ci
            outcome = 'palatability'
            threshold = 0.5;
            task = 'continuous'
            data = X_train

            ## Inner function to embed constraint
            N = data.columns

            intercept = coefficients['intercept'][0]
            intercept_u = coefficients['intercept'][0]
            coeff = coefficients.drop(['intercept'], axis=1, inplace=False).loc['midpoint', :]
            coeff_u = coefficients.drop(['intercept'], axis=1, inplace=False).loc['interval', :]
            # model.add_component('LR'+outcome, 
            #                     Constraint(expr=model.y[outcome] == sum(model.x[i] * coeff.loc[i] for i in N) + intercept))
            model.add_component('LR'+outcome, 
                                Constraint(expr=model.y[outcome] == sum(model.x[i] * coeff.loc[i] for i in N) + intercept
                                          + Γ*(1/len(N)+1)*sum((model.x[i]*model.x[i] * coeff_u.loc[i]**2) for i in N)))

            model.add_component('lb_' + outcome, Constraint(expr=model.y[outcome] >= lb))

            opt = SolverFactory('gurobi')
            opt.options['NonConvex'] = 2
            start_time = time.time()
            results = opt.solve(model) 
            computation_time = time.time() - start_time
            pred_palat = value(model.y['palatability'])
            violation_bool, real_palat = check_violation(threshold,  model.x.get_values())
            ## Save solutions
            solution = model.x.get_values()
            solution['algorithm'] = alg
            solution['iteration'] =i
            solution['price_matrix'] = price_random.to_dict()
            solution['violation'] = violation_bool
            solution['real_palat'] = real_palat
            solution['pred_palat'] = pred_palat
            solution['objective_function'] = value(model.OBJ)
            solution['time'] = computation_time
            solution['max_violation'] = Γ
            solutions_df = solutions_df.append(solution, ignore_index=True)
            print(f"The predicted palatability of the optimal solution is {pred_palat}")



Iteration 0
Generating constraints for the trust region using 1000 samples.
... Trust region defined.
The predicted palatability of the optimal solution is 0.5
Iteration 1
Generating constraints for the trust region using 1000 samples.
... Trust region defined.
The predicted palatability of the optimal solution is 0.5
Iteration 2
Generating constraints for the trust region using 1000 samples.
... Trust region defined.
The predicted palatability of the optimal solution is 0.505200079211692
Iteration 3
Generating constraints for the trust region using 1000 samples.
... Trust region defined.
The predicted palatability of the optimal solution is 0.5818609739262413
Iteration 4
Generating constraints for the trust region using 1000 samples.
... Trust region defined.
The predicted palatability of the optimal solution is 0.5
Iteration 5
Generating constraints for the trust region using 1000 samples.
... Trust region defined.
The predicted palatability of the optimal solution is 0.5
Iteration 6

... Trust region defined.
The predicted palatability of the optimal solution is 0.5
Iteration 0
Generating constraints for the trust region using 1000 samples.
... Trust region defined.
The predicted palatability of the optimal solution is 0.5
Iteration 1
Generating constraints for the trust region using 1000 samples.
... Trust region defined.
The predicted palatability of the optimal solution is 0.5
Iteration 2
Generating constraints for the trust region using 1000 samples.
... Trust region defined.
The predicted palatability of the optimal solution is 0.5591595514396335
Iteration 3
Generating constraints for the trust region using 1000 samples.
... Trust region defined.
The predicted palatability of the optimal solution is 0.6248864880504199
Iteration 4
Generating constraints for the trust region using 1000 samples.
... Trust region defined.
The predicted palatability of the optimal solution is 0.5
Iteration 5
Generating constraints for the trust region using 1000 samples.
... Trust 

In [136]:
solutions_df.to_clipboard()

In [168]:
iterations = 25  # 1000
alg = 'rf'
for viol_limit in [0.1,0.25]:
    print(f"\nAlgorithm: {alg}")
    
    model_master = opticl.model_selection(performance.query('alg == "%s"' % alg), 
                                      constraints_embed, objectives_embed)
    model_master['lb'] = threshold
    model_master['ub'] = None
    opticl.check_model_master(model_master)

    for i in range(iterations):
        print(f"Iteration {i}")
        np.random.seed(i)
        price_random = pd.Series(np.random.random(len(cost_p))*1000)
        price_random.index = cost_p.index

        conceptual_model= init_conceptual_model(price_random)
        MIP_final_model = opticl.optimization_MIP(conceptual_model, conceptual_model.x, model_master, X, 
                                                  tr=True, max_violation = viol_limit)
        opt = SolverFactory('gurobi')
        start_time = time.time()
        results = opt.solve(MIP_final_model) 
        computation_time = time.time() - start_time
        pred_palat = value(MIP_final_model.y['palatability'])
        violation_bool, real_palat = check_violation(threshold,  MIP_final_model.x.get_values())
        ## Save solutions
        solution = MIP_final_model.x.get_values()
        solution['algorithm'] = alg
        solution['iteration'] =i
        solution['price_matrix'] = price_random.to_dict()
        solution['violation'] = violation_bool
        solution['real_palat'] = real_palat
        solution['pred_palat'] = pred_palat
        solution['objective_function'] = value(MIP_final_model.OBJ)
        solution['time'] = computation_time
        solution['max_violation'] = viol_limit
        solutions_df_robust = solutions_df_robust.append(solution, ignore_index=True)
        print(f"The predicted palatability of the optimal solution is {pred_palat}")


Algorithm: rf
        outcome model_type                                  save_path  \
0  palatability         rf  results/rf/TPDP_v1_palatability_model.csv   

         task  objective  
0  continuous          0  
No learned objective

Embedding constraint for palatability using rf model.
0.5 <= palatability
Iteration 0
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.615007043462521
Iteration 1
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.6938226524328955
Iteration 2
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.5823601329902334
Iteration 3
Generating constraints for the trus

... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.6241878122870709
Iteration 11
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.6048265548166682
Iteration 12
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.6328885172139289
Iteration 13
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.5795898713089391
Iteration 14
Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
The predicted palatability of the optimal solution is 0.6328885172139289
It

In [169]:
solutions_df_robust['alg_name'] = solutions_df_robust['algorithm'] + solutions_df_robust['max_violation'].fillna('').astype('str')

In [170]:
solutions_df_robust.groupby('alg_name').agg(mean_pred_palat = ('pred_palat','mean'),
                                           mean_real_palat = ('real_palat','mean'),
                                           mean_violation = ('real_palat',lambda x: (x <= 0.5).mean()),
                                            mean_objective = ('objective_function','mean'))

,mean_pred_palat,mean_real_palat,mean_violation,mean_objective
alg_name,,,,
cart,0.592559,0.519344,0.36,1315.076693
gbm,0.548547,0.519416,0.40,1314.236593
linear,0.517326,0.475888,0.40,1377.951314
rf,0.562095,0.529697,0.36,1322.287131
rf0.1,0.645472,0.546753,0.36,1328.254158
rf0.25,0.616541,0.540756,0.32,1325.859228
rf0.5,0.575758,0.533302,0.36,1323.923979
svm,0.529173,0.522048,0.44,1325.191670


In [201]:
solutions_df.to_csv('results/violations_by_method.csv')